In [1]:
!pip install -r requirements.txt

In [2]:
import gzip
from collections import Counter
from typing import Union

import numpy as np
import pandas as pd
from IPython.display import display  # removes unnecessary error reports in PyCharm
from scipy import sparse
from tqdm import tqdm


tqdm.pandas()

In [3]:
def parse_json(filename: str, read_max: int = None) -> pd.DataFrame:
    """
    Reads the file line by line, parsing each line as json.
    """
    file = gzip.open(filename, "r")
    data = []
    for index, line in enumerate(tqdm(file)):
        if index == read_max:
            break
        line = line.decode()
        line = line.replace("true", "True")
        line = line.replace("false", "False")
        data.append(eval(line))
    print(f"Read {len(data)} rows.")
    return pd.DataFrame.from_dict(data)

# Dataset

The dataset consists of three separate files:

- The books; we will only need this data to make sense of the recommendations. Using this data to make recommendations
is (at least for now) not required for the project. It would require analysis and comparisons of the books, which is not
a part of the base algorithm. However, being able to show which books are being recommended, rather than just showing an
ID, is quite valuable in itself.

- The reviews; this file contains all the reviews and associated ratings that the users have given. This is essentially
the explicit feedback that we can use to generate recommendations.

- The interactions; this file contains all the interactions between users and books. It contains explicit and implicit
feedback, both of which we can use to generate recommendations. We will probably only use the implicit data if we do use
the data in this file.

We have the option of using either explicit (i.e. ratings) or implicit (i.e. interactions) data. We will explore both
options.

The following cells load the data from the files and convert them into the appropriate types. This includes parsing
datetime strings, converting integers to numpy types, etc.

In [4]:
data_path = "data/"
books = "goodreads_books_comics_graphic.json.gz"
interactions = "goodreads_interactions_comics_graphic.json.gz"
reviews = "goodreads_reviews_comics_graphic.json.gz"
datasets = {books: None, interactions: None, reviews: None}

# The maximum number of lines to read from each file
n = 500000

# The filename of the matrix data file (i.e. interactions or reviews) we're currently using
datafile = reviews

for filename in datasets.keys():
    print(filename)
    datasets[filename] = parse_json(data_path + filename, n)
    display(datasets[filename].head(5))

books_df = datasets[books][["book_id", "title"]].copy()
interactions_df = datasets[datafile][["user_id", "book_id", "rating", "date_updated"]].copy()

goodreads_books_comics_graphic.json.gz


89411it [00:38, 2308.70it/s]


Read 89411 rows.


,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,,1,[],US,,"[{'count': '228', 'name': 'to-read'}, {'count'...",B00NLXQ534,True,4.12,,...,,,,https://www.goodreads.com/book/show/25742454-t...,https://s.gr-assets.com/assets/nophoto/book/11...,25742454,1,42749946,The Switchblade Mamma,The Switchblade Mamma
1,2205073346,2,[],US,fre,"[{'count': '2', 'name': 'bd'}, {'count': '2', ...",,False,3.94,,...,1,,2016,https://www.goodreads.com/book/show/30128855-c...,https://images.gr-assets.com/books/1462644346m...,30128855,16,50558228,Cruelle,Cruelle
2,,5,"[246830, 362583, 362581, 623032]",US,eng,"[{'count': '493', 'name': 'to-read'}, {'count'...",,False,4.28,,...,,,2012,https://www.goodreads.com/book/show/13571772-c...,https://images.gr-assets.com/books/1333287305m...,13571772,51,102217,Captain America: Winter Soldier (The Ultimate ...,Captain America: Winter Soldier (The Ultimate ...
3,,1,[],US,eng,"[{'count': '222', 'name': 'to-read'}, {'count'...",B06XKGGSB7,True,4.05,B06XKGGSB7,...,,,,https://www.goodreads.com/book/show/35452242-b...,https://s.gr-assets.com/assets/nophoto/book/11...,35452242,6,54276229,Bounty Hunter 4/3: My Life in Combat from Mari...,Bounty Hunter 4/3: My Life in Combat from Mari...
4,0930289765,6,"[266759, 1096220]",US,en-US,"[{'count': '20', 'name': 'to-read'}, {'count':...",,False,4.06,,...,11,,1997,https://www.goodreads.com/book/show/707611.Sup...,https://images.gr-assets.com/books/1307838888m...,707611,51,693886,"Superman Archives, Vol. 2","Superman Archives, Vol. 2"


goodreads_interactions_comics_graphic.json.gz


...

,user_id,book_id,review_id,is_read,rating,review_text_incomplete,date_added,date_updated,read_at,started_at
0,8842281e1d1347389f2ab93d60773d4d,836610,6b4db26aafeaf0da77c7de6214331e1e,False,0,,Mon Aug 21 12:11:00 -0700 2017,Mon Aug 21 12:11:00 -0700 2017,,
1,8842281e1d1347389f2ab93d60773d4d,7648967,99b27059f711c37de8f90ee8e4dc0d1b,False,0,,Fri Feb 24 08:59:44 -0800 2017,Fri Feb 24 08:59:44 -0800 2017,,
2,8842281e1d1347389f2ab93d60773d4d,15704307,cb944d94854df5afd22210bb0aa0c903,False,0,,Wed May 20 21:28:56 -0700 2015,Wed May 20 21:28:57 -0700 2015,,
3,8842281e1d1347389f2ab93d60773d4d,6902644,2711bac2a8cc600dae1590a6ca0edb34,False,0,,Sun Jun 01 17:25:23 -0700 2014,Sun Jun 01 17:25:23 -0700 2014,,
4,8842281e1d1347389f2ab93d60773d4d,9844623,b72979076d1cded25dded922195e5b1c,False,0,,Sun Sep 02 08:45:08 -0700 2012,Sun Sep 02 08:45:08 -0700 2012,,


goodreads_reviews_comics_graphic.json.gz


500000it [00:19, 25743.26it/s]


Read 500000 rows.


,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,dc3763cdb9b2cae805882878eebb6a32,18471619,66b2ba840f9bd36d6d27f46136fe4772,3,Sherlock Holmes and the Vampires of London \n ...,Thu Dec 05 10:44:25 -0800 2013,Thu Dec 05 10:45:15 -0800 2013,Tue Nov 05 00:00:00 -0800 2013,,0,0
1,bafc2d50014200cda7cb2b6acd60cd73,6315584,72f1229aba5a88f9e72f0dcdc007dd22,4,"I've never really liked Spider-Man. I am, howe...",Wed Aug 10 06:06:48 -0700 2016,Fri Aug 12 08:49:54 -0700 2016,Fri Aug 12 08:49:54 -0700 2016,Wed Aug 10 00:00:00 -0700 2016,0,0
2,bafc2d50014200cda7cb2b6acd60cd73,29847729,a75309355f8662caaa5e2c92ab693d3f,4,"A very quick introduction, this is coming out ...",Thu Apr 21 07:44:00 -0700 2016,Thu Apr 21 07:59:28 -0700 2016,Thu Apr 21 07:59:28 -0700 2016,Thu Apr 21 00:00:00 -0700 2016,0,0
3,bafc2d50014200cda7cb2b6acd60cd73,18454118,c3cc5a3e1d6b6c9cf1c044f306c8e752,5,I've been waiting so long for this. I first st...,Mon Mar 03 17:45:56 -0800 2014,Mon Mar 03 17:54:11 -0800 2014,Sat Mar 01 00:00:00 -0800 2014,Sat Mar 01 00:00:00 -0800 2014,1,0
4,bafc2d50014200cda7cb2b6acd60cd73,2239435,cc444be37ab0a42bfb4dd818cb5edd10,4,The only thing more entertaining than this boo...,Wed Apr 03 12:37:48 -0700 2013,Wed Apr 03 13:03:36 -0700 2013,Wed Apr 03 13:03:36 -0700 2013,,0,0


In [5]:
format_str = "%a %b %d %H:%M:%S %z %Y"
interactions_df["date_updated"] = pd.to_datetime(interactions_df["date_updated"], format=format_str)

books_df["book_id"] = books_df["book_id"].astype("int64")
interactions_df["book_id"] = interactions_df["book_id"].astype("int64")

interactions_df = interactions_df.sort_values(by=["user_id", "date_updated"], ascending=[True, True])

In [6]:
def apply_consecutive_mapping(dataframe: pd.DataFrame, column: str, new_column: str, *additional: pd.DataFrame) -> None:
    """
    Generates a consecutive ID column for the values of an existing column. Also adds this column to additional data
    frames with the exact same mapping of old ID to new (consecutive) ID.
    """
    ids = {}

    def map_to_consecutive_ids(uuid: Union[int, np.int64]) -> int:
        """
        To be used with `pd.Dataframe.apply()` or `pd.Dataframe.progress_apply()`; returns a unique ID per distinct value.
        (Using the function attribute `map_to_consecutive_ids.ids` to avoid namespace pollution.
        """
        if uuid not in ids:
            ids[uuid] = len(ids)
        return ids[uuid]

    dataframe[new_column] = dataframe[column].progress_apply(map_to_consecutive_ids)
    for frame in additional:
        frame[new_column] = frame[column].progress_apply(lambda old_id: ids.get(old_id, -1))


apply_consecutive_mapping(interactions_df, "user_id", "user_id_int")
apply_consecutive_mapping(books_df, "book_id", "book_id_int", interactions_df)

display(books_df.head(10))
display(interactions_df.head(10))

100%|██████████| 500000/500000 [00:00<00:00, 1228083.06it/s]


NameError: name 'ratings_df' is not defined

In [ ]:
def preprocess(df: pd.DataFrame, min_support: int = 5) -> pd.DataFrame:
    """
    Removes reconsumption items, items that have less than `min_support` interactions, and users that have less than
    `min_support` interactions.
    """
    print(df.shape[0], "initial rows")
    # Drop reconsumption items
    df = df.drop_duplicates(subset=["user_id", "item_id"])
    print(df.shape[0], "rows after removing reconsumption items")
    # Compute user and item counts
    g1 = df.groupby("item_id", as_index=False)["user_id"].size()
    g1 = g1.rename({"size": "users_per_item"}, axis="columns")
    g2 = df.groupby("user_id", as_index=False)["item_id"].size()
    g2 = g2.rename({"size": "items_per_user"}, axis="columns")
    df = pd.merge(df, g1, how="left", on=["item_id"])
    df = pd.merge(df, g2, how="left", on=["user_id"])
    # Drop items and users with less than `min_support` interactions
    df = df[df["users_per_item"] >= min_support]
    print(df.shape[0], "rows after removing infrequent items")
    df = df[df["items_per_user"] >= min_support]
    print(df.shape[0], "rows after removing infrequent users")
    df = df[["user_id", "item_id", "datetime", "rating"]].copy()
    return df


interactions_processed = interactions_df[["user_id_int", "book_id_int", "date_updated", "rating"]].copy()
interactions_processed = interactions_processed.rename(
    columns={"user_id_int": "user_id", "book_id_int": "item_id", "date_updated": "datetime"})
display(interactions_processed.head(5))
print(f"Number of unique users:", interactions_processed["user_id"].nunique())
print(f"Number of unique items:", interactions_processed["item_id"].nunique())
interactions_processed = preprocess(interactions_processed)
print(f"Number of unique users:", interactions_processed["user_id"].nunique())
print(f"Number of unique items:", interactions_processed["item_id"].nunique())
display(interactions_processed.head(5))

In [ ]:
def split(items: list[int], percentage_train: float) -> pd.Series:
    nr_train_items = int(len(items) * percentage_train)
    return pd.Series((items[: nr_train_items], items[nr_train_items:]))


sessions_df = interactions_processed.groupby(by="user_id", as_index=False)[["item_id", "datetime", "rating"]].agg(list)
display(sessions_df.head(5))

percentage_train = 0.8
sessions_df[["history", "future"]] = sessions_df["item_id"].progress_apply(split, args=(percentage_train,))
display(sessions_df.head(5))

In [ ]:
def create_sparse_matrix(dataframe: pd.DataFrame, column: str = "history",
                         shape: tuple[int, int] = None) -> sparse.csr_matrix:
    """
    Creates a sparse matrix from the data in `dataframe`.
    """
    # Flatten the dataframe
    user_ids = []
    item_ids = []
    for index, row in dataframe.iterrows():
        user_ids.extend([row["user_id"]] * len(row[column]))
        item_ids.extend(row[column])
    # Create the CSR matrix
    values = np.ones(len(user_ids))
    return sparse.csr_matrix((values, (user_ids, item_ids)), shape=shape, dtype=np.int32)


shape = (interactions_processed["user_id"].max() + 1, interactions_processed["item_id"].max() + 1)
train = create_sparse_matrix(sessions_df, "history", shape)
true = create_sparse_matrix(sessions_df, "future", shape)

In [ ]:
class PopularityRecommender:
    def __init__(self, k: int = 10) -> None:
        self.k = k
        self.scores = []

    def fit(self, data: sparse.csr_matrix) -> None:
        items = list(data.nonzero()[1])
        scores = Counter(items).most_common(self.k)
        self.scores = [(item, score / scores[0][1]) for item, score in scores]

    def predict(self, data: sparse.csr_matrix) -> sparse.csr_matrix:
        items, values = zip(*self.scores)
        users = set(data.nonzero()[0])

        matrix_data = ([], ([], []))
        for user in users:
            matrix_data[0].extend(values)
            matrix_data[1][0].extend([user] * self.k)
            matrix_data[1][1].extend(items)
        return sparse.csr_matrix(matrix_data, shape=data.shape)


k = 20
recommender = PopularityRecommender(k)
recommender.fit(train)
predicted = recommender.predict(train)
print(predicted)

In [ ]:
def sparse_invert_nonzero(a: sparse.csr_matrix) -> sparse.csr_matrix:
    inverse = a.copy()
    inverse.data = 1 / inverse.data
    return inverse


def sparse_divide_nonzero(a: sparse.csr_matrix, b: sparse.csr_matrix) -> sparse.csr_matrix:
    return a.multiply(sparse_invert_nonzero(b))


def compute_recall(true: sparse.csr_matrix, predicted: sparse.csr_matrix) -> float:
    scores = sparse.lil_matrix(predicted.shape)
    scores[predicted.multiply(true).astype(bool)] = 1
    scores = sparse_divide_nonzero(scores.tocsr(), sparse.csr_matrix(true.sum(axis=1))).sum(axis=1)
    return scores.mean()


recall = compute_recall(true, predicted)
print(f"Recall @ {k}: {recall:.4f}")